#### 1.1 Apply Partitioning (Silver Tables)

In [0]:
%sql

CREATE OR REPLACE TABLE medilakehealth_catalog.silver.encounters_clean
USING DELTA
PARTITIONED BY (encounter_date)
AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY encounter_id
           ORDER BY updated_at DESC
         ) AS rn
  FROM medilakehealth_catalog.bronz.encounters
)
WHERE rn = 1;


Exam Rule:
Partition during table creation or full refresh, not incremental MERGE.

##### Lab Results

In [0]:
%sql
CREATE OR REPLACE TABLE medilakehealth_catalog.silver.lab_results_clean
USING DELTA
PARTITIONED BY (result_date)
AS
SELECT *
FROM (
  SELECT *,
         ROW_NUMBER() OVER (
           PARTITION BY lab_result_id
           ORDER BY result_date DESC
         ) AS rn
  FROM medilakehealth_catalog.bronz.lab_results
)
WHERE rn = 1;


#### Z-ORDERING (Second Most Tested)

**When to Z-ORDER?**
- High cardinality column
- Frequently used in filters
- NOT suitable for partitioning

In [0]:
%sql 

OPTIMIZE medilakehealth_catalog.silver.encounters_clean
ZORDER BY (patient_id);


In [0]:
%sql

OPTIMIZE medilakehealth_catalog.silver.lab_results_clean
ZORDER BY (patient_id);



##### OPTIMIZE (Small File Problem)
**Why OPTIMIZE?**
- Bronze ingestion → many small files
- MERGE creates fragmentation

In [0]:
%sql

OPTIMIZE medilakehealth_catalog.silver.encounters_clean;


##### Data Skipping (Hidden Superpower)
**Conditions for Data Skipping**
- Partition pruning
- Z-ORDER
- Predicate pushdown


In [0]:
%sql
SELECT *
FROM medilakehealth_catalog.silver.lab_results_clean
WHERE patient_id = 'P002'
AND result_date >= '2024-12-01';


##### MERGE Performance Tuning (Advanced)
- Large table
- No partition filter

**Bad MERGE**

Exam Tip:
Always restrict MERGE using partition columns if possible.

In [0]:
%sql
MERGE INTO medilakehealth_catalog.silver.encounters_clean tgt
USING updates src
ON tgt.encounter_id = src.encounter_id
AND tgt.encounter_date >= '2024-12-01'
WHEN MATCHED THEN UPDATE SET tgt.updated_at = src.updated_at;